In [ ]:
import $exclude.`org.slf4j:slf4j-log4j12`, $ivy.`org.slf4j:slf4j-nop:1.7.21`
import $profile.`hadoop-2.7`
import $ivy.`org.apache.spark::spark-sql:2.1.0`
import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`
import $ivy.`org.jupyter-scala::spark:0.4.2`
import org.apache.spark._
import org.apache.spark.sql._
import jupyter.spark.session._
val sc = new SparkContext(new SparkConf().setAppName("averageflightdelay").setMaster("local"))

In [2]:
val flight_data = sc.textFile("2007.csv", 2)

flight_data: rdd.RDD[String] = 2007.csv MapPartitionsRDD[1] at textFile at cmd1.sc:1

In [3]:
flight_data.take(4).foreach(println)

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2007,1,1,1,1232,1225,1341,1340,WN,2891,N351,69,75,54,1,7,SMF,ONT,389,4,11,0,,0,0,0,0,0,0
2007,1,1,1,1918,1905,2043,2035,WN,462,N370,85,90,74,8,13,SMF,PDX,479,5,6,0,,0,0,0,0,0,0
2007,1,1,1,2206,2130,2334,2300,WN,1229,N685,88,90,73,34,36,SMF,PDX,479,6,9,0,,0,3,0,0,0,31


In [4]:
val flight_data_filtered = flight_data.filter(line => !line.contains("Year")).map(line => line.split(",").map(elem => elem.trim))

flight_data_filtered: rdd.RDD[Array[String]] = MapPartitionsRDD[3] at map at cmd3.sc:1

In [5]:
println(flight_data_filtered.take(4).deep)

Array(Array(2007, 1, 1, 1, 1232, 1225, 1341, 1340, WN, 2891, N351, 69, 75, 54, 1, 7, SMF, ONT, 389, 4, 11, 0, , 0, 0, 0, 0, 0, 0), Array(2007, 1, 1, 1, 1918, 1905, 2043, 2035, WN, 462, N370, 85, 90, 74, 8, 13, SMF, PDX, 479, 5, 6, 0, , 0, 0, 0, 0, 0, 0), Array(2007, 1, 1, 1, 2206, 2130, 2334, 2300, WN, 1229, N685, 88, 90, 73, 34, 36, SMF, PDX, 479, 6, 9, 0, , 0, 3, 0, 0, 0, 31), Array(2007, 1, 1, 1, 1230, 1200, 1356, 1330, WN, 1355, N364, 86, 90, 75, 26, 30, SMF, PDX, 479, 3, 8, 0, , 0, 23, 0, 0, 0, 3))


In [6]:
import scala.util.Try

import scala.util.Try

In [7]:
var arrivalTuple = flight_data_filtered.filter(rec=>Try(rec(14).toDouble).isSuccess).map(rec=>(rec(3).toInt,rec(14).toDouble))

arrivalTuple: rdd.RDD[(Int, Double)] = MapPartitionsRDD[5] at map at cmd6.sc:1

In [8]:
arrivalTuple.take(4).foreach(println)

(1,1.0)
(1,8.0)
(1,34.0)
(1,26.0)


In [9]:
val arrivalDelayByWeekDay = arrivalTuple.mapValues(value => (value,1)).reduceByKey((accumulated,current)=>(accumulated._1 + current._1,accumulated._2 + current._2)).mapValues(value => 1.0 * value._1 /value._2).collect

arrivalDelayByWeekDay: Array[(Int, Double)] = Array(
  (4, 3.848097317529632),
  (6, -1.8044743543106585),
  (2, 0.1321186881832379),
  (1, 4.845759130697253),
  (3, 1.3002536461636018),
  (7, 3.357736516357206),
  (5, 7.87126000842815)
)

### Average Arrival Delay Per Day of Week (in minute)

In [14]:
arrivalDelayByWeekDay.sortBy(value => value).foreach(println)

(1,4.845759130697253)
(2,0.1321186881832379)
(3,1.3002536461636018)
(4,3.848097317529632)
(5,7.87126000842815)
(6,-1.8044743543106585)
(7,3.357736516357206)


In [10]:
var delayTuple = flight_data_filtered.filter(rec=>Try(rec(14).toDouble).isSuccess).map(rec=>(rec(3).toInt,rec(15).toDouble))

delayTuple: rdd.RDD[(Int, Double)] = MapPartitionsRDD[10] at map at cmd9.sc:1

In [11]:
delayTuple.take(4).foreach(println)

(1,7.0)
(1,13.0)
(1,36.0)
(1,30.0)


In [12]:
val departueDelayByWeekDay = delayTuple.mapValues(value => (value,1)).reduceByKey((accumulated, current) => (accumulated._1 + current._1,accumulated._2 + current._2)).mapValues(value => 1.0 * value._1 /value._2).collect

departueDelayByWeekDay: Array[(Int, Double)] = Array(
  (4, 8.430338947806197),
  (6, 5.727537286285922),
  (2, 6.195627277459656),
  (1, 10.058406278297616),
  (3, 7.054745296977384),
  (7, 9.41945181255526),
  (5, 12.41361146228403)
)

### Average Departure Delay Per Day of Week (in minute)

In [13]:
departueDelayByWeekDay.sortBy(value => value).foreach(println)

(1,10.058406278297616)
(2,6.195627277459656)
(3,7.054745296977384)
(4,8.430338947806197)
(5,12.41361146228403)
(6,5.727537286285922)
(7,9.41945181255526)
